# 0. basic settings

In [ ]:
import numpy as np
import pandas as pd
import statsmodels.api as sm
import statsmodels.formula.api as smf
import seaborn as sns
import matplotlib.pyplot as plt
import datetime
import difflib
import shapefile  
import math

from sklearn.linear_model import LassoCV
from sklearn import linear_model
from sklearn import linear_model
from random import sample


In [ ]:
# Functions
def Loc(x,y,a,b):
    # Enter two pairs of longitude and latitude to restore the horizontal and vertical distance
    x1 = (a-x)/180*np.pi*meter_R
    y1 = (b-y)/180*np.pi*meter_R 
    
    return x1,y1

In [ ]:
# Parameters
dataInDIR = # where your raw data stores
dataSaveDIR = # where you want to save your derived data

etyName2018 = # 2018 entry data file
etyName2019 = # 2019 entry data file

etylat = #entry latitude label name
etylon = #entry longitude label name
etyopendate = #entry openning date label name

mile_R = 3963.191 # earth radius in miles
meter_R = 6378137 # earth radius in meters

# feature choosing range
      # acs
M_delta_radious = # Criteria for raw data, within 5 miles
M_delta_degrees = math.degrees(M_delta_radious)

    # ety-acs
M_delta_radious2 = 5/mile_R 
M_delta_degrees2 = math.degrees(M_delta_radious2)

    # ety feature
M_delta_radious3 = 10/mile_R 
M_delta_degrees3 = math.degrees(M_delta_radious3)

    # poi feature
p_delta_radious = 5/mile_R 
p_delta_degrees = math.degrees(p_delta_radious)


# 1. Entry

In [ ]:
# load in data
ety2018 = pd.read_stata(dataInDIR+etyName2018)  
ety2019 = pd.read_stata(dataInDIR+etyName2019)  

ety2018 = ety2018.loc[:,['opening_placekey','opening_latitude','opening_longitude','opening_month']]
ety2018.columns = ['sg_placekey', etylat,etylon,etyopendate]

ety2019 = ety2019.loc[:,['opening_placekey','opening_lat','opening_lon','opening_mon']]
ety2019.columns = ['sg_placekey', etylat,etylon,etyopendate]

ety2018.insert(loc=0,column='No',value=range(len(ety2018.index)),allow_duplicates=True)
ety2019.insert(loc=0,column='No',value=range(len(ety2019.index)),allow_duplicates=True)


In [ ]:
entryMon = ety2018[etyopendate].copy()
etyMon = []

for item in entryMon:

    item = int(float(item))
    if item<=12:
        temp = datetime.datetime(2018, item, 1, 0, 0, 0)
    elif item<=24:
        temp = datetime.datetime(2019, item-12, 1, 0, 0, 0)

    etyMon.append(temp)
    
ety2018[etyopendate] = etyMon

entryMon = ety2019[etyopendate].copy()
etyMon = []

for item in entryMon:
    item = int(float(item))
    if item<=12:
        temp = datetime.datetime(2018, item, 1, 0, 0, 0)
    elif item<=24:
        temp = datetime.datetime(2019, item-12, 1, 0, 0, 0)

    etyMon.append(temp)
ety2019[etyopendate] = etyMon

In [ ]:
ety = ety2018.append(ety2019)
ety['No'] = range(len(ety.index))
ety.index = range(len(ety.index))


# 2. Feature ACS

In [ ]:
# load in data
acs1 = pd.read_stata(dataInDIR+filenameACSdata)  

# basic processing
acs2 = acs1[['population','share_age_under16', 'share_age_16_64','share_white','share_college', 'share_collegeplus',
            'median_hhinc','share_unemployed', 'median_hh_value','share_trans_car', 'share_trans_public', 'share_trans_home',  
'share_travel_0_15', 'share_travel_15_30', 'share_travel_30_45', 'share_travel_45_60', 'share_travel_60_90', 'cbg_id'
 ]]
minorRace = 1-acs1['share_white']
acs2['minorRace'] = minorRace

In [ ]:
# latitude and longitude
# read in CBG data of USA
state = 'US'
fname =dataInDIR +'{}.txt'.format(state)
dataST = pd.read_csv(fname)

# add FIPS label
lendataST = len(dataST.index)
FIPSs = []
for i in range(lendataST):
    if i%5000==0:
        print(i)
    tractce1 = str(dataST.loc[i,'STATEFP'])
    tractce1 = (2-len(tractce1))*'0'+tractce1
    tractce2 = str(dataST.loc[i,'COUNTYFP'])
    tractce2 = (3-len(tractce2))*'0'+tractce2
    tractce3 = str(dataST.loc[i,'TRACTCE'])
    tractce3 = (6-len(tractce3))*'0'+tractce3
    tractce4 = str(dataST.loc[i,'BLKGRPCE'])
    tractce4 = (1-len(tractce4))*'0'+tractce4
    tractce = tractce1+tractce2+tractce3+tractce4
    #print(tractce)
    FIPSs.append(tractce)

dataST['cbg_id'] = FIPSs


In [ ]:
# Fusing block information to get latitude and longitude
acs3 = pd.merge(acs2,dataST,on='cbg_id',how='left')

# screen outliers
acs4 = acs3.copy()
acs4 = acs4.drop_duplicates(subset=['cbg_id']) # the amounts keeps the same after dropping duplication
acs4 = acs4.dropna(axis = 0,how='any')
acs4.index = range(len(acs4.index))


## 2.1 Delineate the grid scope

In [ ]:
# Filter acs points based on entry perimeter

acs5 = acs4.copy()
cnt = 0
cnt0 = 0
for i in range(len(ety.index)):
    lat = ety.loc[i,etylat]
    lon = ety.loc[i,etylon]

    temp = acs5[acs5['LATITUDE']>=lat-M_delta_degrees].copy()
    temp = temp[temp['LATITUDE']<=lat+M_delta_degrees]
    temp = temp[temp['LONGITUDE']>=lon-M_delta_degrees]
    temp = temp[temp['LONGITUDE']<=lon+M_delta_degrees]
    cnt+=len(temp.index)
    
    if len(temp.index)==0:
        cnt0+=1
        
    if i==0:
        outcome = temp
    else:
        outcome = outcome.append(temp)

print('cnt',cnt,'entry with no surroundings',cnt0)
outcome = outcome.drop_duplicates(subset=['cbg_id'])
acs6 = outcome.copy()
acs6.index = range(len(acs6.index))
acs6.insert(loc=0,column='No',value=range(len(acs6.index)),allow_duplicates=True)


In [ ]:
# grid_S_S

grid_S_S = []
ety2 = ety.copy(deep=True)

for i in range(len(ety.index)):
    if i%50==0:
        print(i)
        
    lat = ety.loc[i,etylat]
    lon = ety.loc[i,etylon]
    temp = ety2[ety2[etylat]>=lat-M_delta_degrees3].copy()
    temp = temp[temp[etylat]<=lat+M_delta_degrees3]
    temp = temp[temp[etylon]>=lon-M_delta_degrees3]
    temp = temp[temp[etylon]<=lon+M_delta_degrees3]
    #samp = sample(range(nbct), 200)
    for j in temp.index:
        xt,yt = Loc(lat,lon,temp.loc[j,etylat],temp.loc[j,etylon])
        line = [ety.loc[i,'No'],temp.loc[j,'No'],xt,yt]
        grid_S_S.append(line)
temp = np.array(grid_S_S)
out = pd.DataFrame(data=temp, index=None, columns=['S_id','S_id_oth','x','y'])

# save data

out.to_csv(dataSaveDIR+'grid_S_S.csv'
           ,index=False,header=out.columns,sep=',')

In [ ]:
# grid_S_I
grid_S_I = []
for i in range(len(ety.index)):
    print(i)
    lat = ety.loc[i,etylat]
    lon = ety.loc[i,etylon]
    temp = acs6[acs6['LATITUDE']>=lat-M_delta_degrees2].copy()
    temp = temp[temp['LATITUDE']<=lat+M_delta_degrees2]
    temp = temp[temp['LONGITUDE']>=lon-M_delta_degrees2]
    temp = temp[temp['LONGITUDE']<=lon+M_delta_degrees2]
    #samp = sample(range(nbct), 200)
    for j in temp.index:
        xt,yt = Loc(lat,lon,temp.loc[j,'LATITUDE'],temp.loc[j,'LONGITUDE'])
        line = [ety.loc[i,'No'],temp.loc[j,'No'],xt,yt,temp.loc[j,'population']
                                                        ,temp.loc[j,'share_age_under16']
                                                        ,temp.loc[j,'share_age_16_64']
                                                        ,temp.loc[j,'share_white']
                                                        ,temp.loc[j,'share_college']
                                                        ,temp.loc[j,'share_collegeplus']
                                                        ,temp.loc[j,'median_hhinc']
                                                        ,temp.loc[j,'share_unemployed']
                                                        ,temp.loc[j,'median_hh_value']
                                                        ,temp.loc[j,'share_trans_car']
                                                        ,temp.loc[j,'share_trans_public']
                                                        ,temp.loc[j,'share_trans_home']
                                                        ,temp.loc[j,'share_travel_0_15']
                                                        ,temp.loc[j,'share_travel_15_30']
                                                        ,temp.loc[j,'share_travel_30_45']
                                                        ,temp.loc[j,'share_travel_45_60']
                                                        ,temp.loc[j,'share_travel_60_90']
                                                        ,temp.loc[j,'cbg_id']
                                                        ,temp.loc[j,'minorRace']]
        grid_S_I.append(line)
temp = np.array(grid_S_I)
out = pd.DataFrame(data=temp, index=None, columns=['S_id','S_id_oth','x','y','population', 'share_age_under16', 'share_age_16_64',
       'share_white', 'share_college', 'share_collegeplus', 'median_hhinc',
       'share_unemployed', 'median_hh_value', 'share_trans_car',
       'share_trans_public', 'share_trans_home', 'share_travel_0_15',
       'share_travel_15_30', 'share_travel_30_45', 'share_travel_45_60',
       'share_travel_60_90', 'cbg_id', 'minorRace'])

# save data

out.to_csv(dataSaveDIR+'grid_S_I.csv'
           ,index=False,header=out.columns,sep=',')

In [ ]:
# Random select
sampS = sample(range(len(acs6.index)),1000) 

sampS = np.array(sampS)
np.savetxt(dataSaveDIR+'sampS.txt',sampS)

# make training data for counterfactual locations

grid_S_random_I = [] # Random points to the location of 200 restaurants
grid_S_random_S = [] # Random points to the location of all real retail stores

ety2 = ety.copy(deep=True)

cnt=1000 

for i in sampS:
    cnt+=1
    print(cnt)
    lat = acs6.loc[i,'LATITUDE']
    lon = acs6.loc[i,'LONGITUDE']
    temp = acs6[acs6['LATITUDE']>=lat-M_delta_degrees2].copy() 
    temp = temp[temp['LATITUDE']<=lat+M_delta_degrees2]
    temp = temp[temp['LONGITUDE']>=lon-M_delta_degrees2]
    temp = temp[temp['LONGITUDE']<=lon+M_delta_degrees2]
    #samp = sample(range(nbct), 200)
    for j in temp.index:
        xt,yt = Loc(lat,lon,temp.loc[j,'LATITUDE'],temp.loc[j,'LONGITUDE'])
        line = [cnt,temp.loc[j,'No'],xt,yt,temp.loc[j,'population']
                                                        ,temp.loc[j,'share_age_under16']
                                                        ,temp.loc[j,'share_age_16_64']
                                                        ,temp.loc[j,'share_white']
                                                        ,temp.loc[j,'share_college']
                                                        ,temp.loc[j,'share_collegeplus']
                                                        ,temp.loc[j,'median_hhinc']
                                                        ,temp.loc[j,'share_unemployed']
                                                        ,temp.loc[j,'median_hh_value']
                                                        ,temp.loc[j,'share_trans_car']
                                                        ,temp.loc[j,'share_trans_public']
                                                        ,temp.loc[j,'share_trans_home']
                                                        ,temp.loc[j,'share_travel_0_15']
                                                        ,temp.loc[j,'share_travel_15_30']
                                                        ,temp.loc[j,'share_travel_30_45']
                                                        ,temp.loc[j,'share_travel_45_60']
                                                        ,temp.loc[j,'share_travel_60_90']
                                                        ,temp.loc[j,'cbg_id']
                                                        ,temp.loc[j,'minorRace']]
        grid_S_random_I.append(line)
        
        
    temp = ety2[ety2[etylat]>=lat-M_delta_degrees3].copy()
    temp = temp[temp[etylat]<=lat+M_delta_degrees3]
    temp = temp[temp[etylon]>=lon-M_delta_degrees3]
    temp = temp[temp[etylon]<=lon+M_delta_degrees3]
    #samp = sample(range(nbct), 200)
    for j in temp.index:
        xt,yt = Loc(lat,lon,temp.loc[j,etylat],temp.loc[j,etylon])
        line = [cnt,temp.loc[j,'No'],xt,yt]
        grid_S_random_S.append(line)
        
temp = np.array(grid_S_random_I)
out = pd.DataFrame(data=temp, index=None, columns=['S_id','S_id_oth','x','y','population', 'share_age_under16', 'share_age_16_64',
       'share_white', 'share_college', 'share_collegeplus', 'median_hhinc',
       'share_unemployed', 'median_hh_value', 'share_trans_car',
       'share_trans_public', 'share_trans_home', 'share_travel_0_15',
       'share_travel_15_30', 'share_travel_30_45', 'share_travel_45_60',
       'share_travel_60_90', 'cbg_id', 'minorRace'])

temp2 = np.array(grid_S_random_S)
out2 = pd.DataFrame(data=temp2, index=None, columns=['S_id','S_id_oth','x','y'])

# save data
out.to_csv(dataSaveDIR+'grid_S_random_I.csv'
           ,index=False,header=out.columns,sep=',')
out2.to_csv(dataSaveDIR+'grid_S_random_S.csv'
           ,index=False,header=out2.columns,sep=',')


# 3. Industry Distribution Feature 

In [ ]:
# load in data
poi1 = pd.read_csv(dataInDIR + POIfileName)
# number of POIs
poi1 = poi1.drop_duplicates(subset=['placekey'])


In [ ]:

poi2 = [] # Random points to the location of 200 restaurants

#cnt=1000
for i in ety.index:
    print(i)
    lat = ety.loc[i,etylat]
    lon = ety.loc[i,etylon]
    temp = poi1[poi1['lat']>=lat-p_delta_degrees].copy() 
    temp = temp[temp['lat']<=lat+p_delta_degrees]
    temp = temp[temp['lon']>=lon-p_delta_degrees]
    temp = temp[temp['lon']<=lon+p_delta_degrees]
    for j in temp.index:
        line = [temp.loc[j,'placekey'],temp.loc[j,'start_date'],temp.loc[j,'end_date'],
                float(temp.loc[j,'naics']),float(temp.loc[j,'lon']),float(temp.loc[j,'lat']),
                float(ety.loc[i,'No']), float(ety.loc[i,etylat]),float(ety.loc[i,etylon]), 
                ety.loc[i,etyopendate]
                ]  

        poi2.append(line)
        

    

In [ ]:
temp = poi2.copy()
poi3 = pd.DataFrame(data=temp, index=None, columns=['poi_placekey', 'start_date', 'end_date', 'poi_naics',
       'poi_lon', 'poi_lat', 'entry_number', 'entry_opening_lat', 'entry_opening_lon', 'entry_opening_date'
       ])

## 3.1. 5 Catogories + 1 Competitors

In [ ]:
# adding start and end day for POIs (in datetime dataform)

poi4 = poi3.copy(deep=True)
startdate = []
endate = []

for i in range(len(poi3.index)):
    
    temp = poi4['start_date'][i]
    temp1 = datetime.datetime.strptime(temp,'%Y-%m-%d')
    startdate.append(temp1)
    
    temp = poi4['end_date'][i]
    temp1 = datetime.datetime.strptime(temp,'%Y-%m-%d')
    endate.append(temp1)
    
poi4['poiStartDate'] = startdate
poi4['poiEndDate'] = endate

In [ ]:
# adding opening datetime for entry openings
poi5 = poi4.copy(deep=True)
entryMon = poi5['entry_opening_date'].copy()

poi5['entryOpenDate'] = entryMon

In [ ]:
# filter entry openings according to opening month
poi6 = poi5.copy(deep=True)


In [ ]:
# five categories
poi7 = poi6.copy(deep=True)
shortNaics = poi7['poi_naics'].copy()
shortNaics = np.array([float(item) for item in shortNaics])
shortNaics = np.floor(shortNaics/10000)

poi7['shortPoiNaics'] = shortNaics

# category 1
cate1 = poi7[poi7['shortPoiNaics']>=52].copy(deep=True)
cate1 = cate1[cate1['shortPoiNaics']<=56]

#cate1

# category 2
cate2 = poi7[poi7['shortPoiNaics']==62].copy(deep=True)

# category 3
cate3 = poi7[poi7['shortPoiNaics']==72].copy(deep=True)

# category 4
cate41 = poi7[poi7['shortPoiNaics']==42].copy(deep=True)
cate42 = poi7[poi7['shortPoiNaics']==44].copy(deep=True)
cate43 = poi7[poi7['shortPoiNaics']==45].copy(deep=True)

cate4 = cate41.append(cate42).copy()
cate4 = cate4.append(cate43)

# category 5
cate51 = poi7[poi7['shortPoiNaics']==51].copy(deep=True)
cate52 = poi7[poi7['shortPoiNaics']==61].copy(deep=True)
cate53 = poi7[poi7['shortPoiNaics']==71].copy(deep=True)
cate54 = poi7[poi7['shortPoiNaics']==81].copy(deep=True)
cate55 = poi7[poi7['shortPoiNaics']==92].copy(deep=True)

cate5 = cate51.append(cate52).copy()
cate5 = cate5.append(cate53)
cate5 = cate5.append(cate54)
cate5 = cate5.append(cate55)

cate1.to_csv(dataSaveDIR+'F3cate1.csv',
           index=False,header=cate1.columns,sep=',')
cate2.to_csv(dataSaveDIR+'F3cate2.csv',
           index=False,header=cate2.columns,sep=',')
cate3.to_csv(dataSaveDIR+'F3cate3.csv',
           index=False,header=cate3.columns,sep=',')
cate4.to_csv(dataSaveDIR+'F3cate4.csv',
           index=False,header=cate4.columns,sep=',')
cate5.to_csv(dataSaveDIR+'F3cate5.csv',
           index=False,header=cate5.columns,sep=',')

In [ ]:
# competitors
poi8 = poi6.copy(deep=True)
sixDNaics = poi8['poi_naics'].copy()
sixDNaics = np.array([float(item) for item in sixDNaics])
sixDNaics = np.floor(sixDNaics)

poi8['6digitPoiNaics'] = sixDNaics

comp1 = poi8[poi8['6digitPoiNaics']==445110].copy(deep=True)
comp2 = poi8[poi8['6digitPoiNaics']==445120].copy(deep=True)
comp3 = poi8[poi8['6digitPoiNaics']==452210].copy(deep=True)
comp4 = poi8[poi8['6digitPoiNaics']==452319].copy(deep=True)

comp = comp1.append(comp2).copy()
comp = comp.append(comp3)
comp = comp.append(comp4)

comp.to_csv(dataSaveDIR+'F3comp.csv',
           index=False,header=comp.columns,sep=',')

In [ ]:
# constucting inputs
grid_S_I_cate1 = []
for i in range(len(ety.index)):
    print(i)
    lat = ety.loc[i,etylat]
    lon = ety.loc[i,etylon]
    temp = cate1[cate1['poi_lat']>=lat-p_delta_degrees].copy()
    temp = temp[temp['poi_lat']<=lat+p_delta_degrees]
    temp = temp[temp['poi_lon']>=lon-p_delta_degrees]
    temp = temp[temp['poi_lon']<=lon+p_delta_degrees]
    #samp = sample(range(nbct), 200)
    for j in temp.index:
        xt,yt = Loc(lat,lon,temp.loc[j,'poi_lat'],temp.loc[j,'poi_lon'])
        line = [ety.loc[i,'No'],j,xt,yt,temp.loc[j,'poi_naics']]
        grid_S_I_cate1.append(line)
        
temp = np.array(grid_S_I_cate1)
out = pd.DataFrame(data=temp, index=None, columns=['S_id','S_id_cate1','x','y','naics'])

# save data
out.to_csv(dataSaveDIR+'grid_S_I_cate1.csv'
           ,index=False,header=out.columns,sep=',')

In [ ]:
grid_S_I_cate2 = []
for i in range(len(ety.index)):
    print(i)
    lat = ety.loc[i,etylat]
    lon = ety.loc[i,etylon]
    temp = cate2[cate2['poi_lat']>=lat-p_delta_degrees].copy()
    temp = temp[temp['poi_lat']<=lat+p_delta_degrees]
    temp = temp[temp['poi_lon']>=lon-p_delta_degrees]
    temp = temp[temp['poi_lon']<=lon+p_delta_degrees]
    #samp = sample(range(nbct), 200)
    for j in temp.index:
        xt,yt = Loc(lat,lon,temp.loc[j,'poi_lat'],temp.loc[j,'poi_lon'])
        line = [ety.loc[i,'No'],j,xt,yt,temp.loc[j,'poi_naics']]
        grid_S_I_cate2.append(line)
        
temp = np.array(grid_S_I_cate2)
out = pd.DataFrame(data=temp, index=None, columns=['S_id','S_id_cate2','x','y','naics'])

# save data
out.to_csv(dataSaveDIR+'grid_S_I_cate2.csv'
           ,index=False,header=out.columns,sep=',')

In [ ]:
grid_S_I_cate3 = []
for i in range(len(ety.index)):
    print(i)
    lat = ety.loc[i,etylat]
    lon = ety.loc[i,etylon]
    temp = cate3[cate3['poi_lat']>=lat-p_delta_degrees].copy()
    temp = temp[temp['poi_lat']<=lat+p_delta_degrees]
    temp = temp[temp['poi_lon']>=lon-p_delta_degrees]
    temp = temp[temp['poi_lon']<=lon+p_delta_degrees]
    #samp = sample(range(nbct), 200)
    for j in temp.index:
        xt,yt = Loc(lat,lon,temp.loc[j,'poi_lat'],temp.loc[j,'poi_lon'])
        line = [ety.loc[i,'No'],j,xt,yt,temp.loc[j,'poi_naics']]
        grid_S_I_cate3.append(line)
        
temp = np.array(grid_S_I_cate3)
out = pd.DataFrame(data=temp, index=None, columns=['S_id','S_id_cate3','x','y','naics'])

# save data
out.to_csv(dataSaveDIR+'grid_S_I_cate3.csv'
           ,index=False,header=out.columns,sep=',')

In [ ]:
grid_S_I_cate4 = []
for i in range(len(ety.index)):
    print(i)
    lat = ety.loc[i,etylat]
    lon = ety.loc[i,etylon]
    temp = cate4[cate4['poi_lat']>=lat-p_delta_degrees].copy()
    temp = temp[temp['poi_lat']<=lat+p_delta_degrees]
    temp = temp[temp['poi_lon']>=lon-p_delta_degrees]
    temp = temp[temp['poi_lon']<=lon+p_delta_degrees]
    #samp = sample(range(nbct), 200)
    for j in temp.index:
        xt,yt = Loc(lat,lon,temp.loc[j,'poi_lat'],temp.loc[j,'poi_lon'])
        line = [ety.loc[i,'No'],j,xt,yt,temp.loc[j,'poi_naics']]
        grid_S_I_cate4.append(line)
        
temp = np.array(grid_S_I_cate4)
out = pd.DataFrame(data=temp, index=None, columns=['S_id','S_id_cate4','x','y','naics'])

# save data
out.to_csv(dataSaveDIR+'grid_S_I_cate4.csv'
           ,index=False,header=out.columns,sep=',')

In [ ]:
grid_S_I_cate5 = []
for i in range(len(ety.index)):
    print(i)
    lat = ety.loc[i,etylat]
    lon = ety.loc[i,etylon]
    temp = cate5[cate5['poi_lat']>=lat-p_delta_degrees].copy()
    temp = temp[temp['poi_lat']<=lat+p_delta_degrees]
    temp = temp[temp['poi_lon']>=lon-p_delta_degrees]
    temp = temp[temp['poi_lon']<=lon+p_delta_degrees]
    #samp = sample(range(nbct), 200)
    for j in temp.index:
        xt,yt = Loc(lat,lon,temp.loc[j,'poi_lat'],temp.loc[j,'poi_lon'])
        line = [ety.loc[i,'No'],j,xt,yt,temp.loc[j,'poi_naics']]
        grid_S_I_cate5.append(line)
        
temp = np.array(grid_S_I_cate5)
out = pd.DataFrame(data=temp, index=None, columns=['S_id','S_id_cate5','x','y','naics'])

# save data
out.to_csv(dataSaveDIR+'grid_S_I_cate5.csv'
           ,index=False,header=out.columns,sep=',')

In [ ]:
#comp
grid_S_I_comp = []
for i in range(len(ety.index)):
    print(i)
    lat = ety.loc[i,etylat]
    lon = ety.loc[i,etylon]
    temp = comp[comp['poi_lat']>=lat-p_delta_degrees].copy()
    temp = temp[temp['poi_lat']<=lat+p_delta_degrees]
    temp = temp[temp['poi_lon']>=lon-p_delta_degrees]
    temp = temp[temp['poi_lon']<=lon+p_delta_degrees]
    #samp = sample(range(nbct), 200)
    for j in temp.index:
        xt,yt = Loc(lat,lon,temp.loc[j,'poi_lat'],temp.loc[j,'poi_lon'])
        line = [ety.loc[i,'No'],j,xt,yt,temp.loc[j,'poi_naics']]
        grid_S_I_comp.append(line)
        
temp = np.array(grid_S_I_comp)
out = pd.DataFrame(data=temp, index=None, columns=['S_id','S_id_comp','x','y','naics'])

# save data
out.to_csv(dataSaveDIR+'grid_S_I_comp.csv'
           ,index=False,header=out.columns,sep=',')

# 4. Counterfactual Inputs

In [ ]:
# for S_random

pr_delta_radious = 5/mile_R # Michael's criteria for raw data, within 5 miles parameter!!!
pr_delta_degrees = math.degrees(pr_delta_radious)

poi2r = [] # Random points to the location of 200 restaurants

#cnt=1000
for i in sampS:
    lat = acs6.loc[i,'LATITUDE']
    lon = acs6.loc[i,'LONGITUDE']
    temp = poi1[poi1['lat']>=lat-pr_delta_degrees].copy() 
    temp = temp[temp['lat']<=lat+pr_delta_degrees]
    temp = temp[temp['lon']>=lon-pr_delta_degrees]
    temp = temp[temp['lon']<=lon+pr_delta_degrees]
    #samp = sample(range(nbct), 200)
    for j in temp.index:
        line = [temp.loc[j,'placekey'],temp.loc[j,'start_date'],temp.loc[j,'end_date'],
                float(temp.loc[j,'naics']),float(temp.loc[j,'lon']),float(temp.loc[j,'lat']),
                float(acs6.loc[i,'No']),lat,lon,
                ]
        poi2r.append(line)

In [ ]:
temp = poi2r.copy()
poi3r = pd.DataFrame(data=temp, index=None, columns=['poi_placekey', 'start_date', 'end_date', 'poi_naics',
       'poi_lon', 'poi_lat', 'entry_number', 'entry_opening_lat', 'entry_opening_lon',
                                                    ])
poi3r = poi3r.drop_duplicates(subset=['poi_placekey'])
poi3r.index = range(len(poi3r.index))


In [ ]:
# adding start and end day for POIs (in datetime dataform)

poi4r = poi3r.copy(deep=True)
startdate = []
endate = []

for i in range(len(poi3r.index)):
    if i%10000==0:
        print(i)
    temp = poi4r['start_date'][i]
    temp1 = datetime.datetime.strptime(temp,'%Y-%m-%d')
    startdate.append(temp1)
    
    temp = poi4r['end_date'][i]
    temp1 = datetime.datetime.strptime(temp,'%Y-%m-%d')
    endate.append(temp1)
    
poi4r['poiStartDate'] = startdate
poi4r['poiEndDate'] = endate


In [ ]:
# adding opening datetime for entry openings
poi5r = poi4r.copy(deep=True)
etyMon = []

for item in range(len(poi4r.index)):

    temp = datetime.datetime(2019, 12, 1, 0, 0, 0)

    etyMon.append(temp)
    
poi5r['entryOpenMon'] = etyMon


In [ ]:
poi6r = poi3r.copy(deep=True)

# five categories
poi7r = poi6r.copy(deep=True)
shortNaics = poi7r['poi_naics'].copy()
shortNaics = np.array([float(item) for item in shortNaics])
shortNaics = np.floor(shortNaics/10000)
#shortNaics = np.int(shortNaics)
#shortNaics
poi7r['shortPoiNaics'] = shortNaics

# category 1
cate1 = poi7r[poi7r['shortPoiNaics']>=52].copy(deep=True)
cate1 = cate1[cate1['shortPoiNaics']<=56]

#cate1

# category 2
cate2 = poi7r[poi7r['shortPoiNaics']==62].copy(deep=True)

# category 3
cate3 = poi7r[poi7r['shortPoiNaics']==72].copy(deep=True)

# category 4
cate41 = poi7r[poi7r['shortPoiNaics']==42].copy(deep=True)
cate42 = poi7r[poi7r['shortPoiNaics']==44].copy(deep=True)
cate43 = poi7r[poi7r['shortPoiNaics']==45].copy(deep=True)

cate4 = cate41.append(cate42).copy()
cate4 = cate4.append(cate43)

# category 5
cate51 = poi7r[poi7r['shortPoiNaics']==51].copy(deep=True)
cate52 = poi7r[poi7r['shortPoiNaics']==61].copy(deep=True)
cate53 = poi7r[poi7r['shortPoiNaics']==71].copy(deep=True)
cate54 = poi7r[poi7r['shortPoiNaics']==81].copy(deep=True)
cate55 = poi7r[poi7r['shortPoiNaics']==92].copy(deep=True)

cate5 = cate51.append(cate52).copy()
cate5 = cate5.append(cate53)
cate5 = cate5.append(cate54)
cate5 = cate5.append(cate55)

cate1.to_csv(dataSaveDIR+'F3cate1r.csv',
           index=False,header=cate1.columns,sep=',')
cate2.to_csv(dataSaveDIR+'F3cate2r.csv',
           index=False,header=cate2.columns,sep=',')
cate3.to_csv(dataSaveDIR+'F3cate3r.csv',
           index=False,header=cate3.columns,sep=',')
cate4.to_csv(dataSaveDIR+'F3cate4r.csv',
           index=False,header=cate4.columns,sep=',')
cate5.to_csv(dataSaveDIR+'F3cate5r.csv',
           index=False,header=cate5.columns,sep=',')

In [ ]:
# competitors
poi8r = poi6r.copy(deep=True)
sixDNaics = poi8r['poi_naics'].copy()
sixDNaics = np.array([float(item) for item in sixDNaics])
sixDNaics = np.floor(sixDNaics)

poi8r['6digitPoiNaics'] = sixDNaics

comp1 = poi8r[poi8r['6digitPoiNaics']==445110].copy(deep=True)
comp2 = poi8r[poi8r['6digitPoiNaics']==445120].copy(deep=True)
comp3 = poi8r[poi8r['6digitPoiNaics']==452210].copy(deep=True)
comp4 = poi8r[poi8r['6digitPoiNaics']==452319].copy(deep=True)

comp = comp1.append(comp2).copy()
comp = comp.append(comp3)
comp = comp.append(comp4)

comp.to_csv(dataSaveDIR+'F3compr.csv',
           index=False,header=comp.columns,sep=',')

In [ ]:
grid_S_random_I_cate1 = [] #
cnt=1000 
for i in sampS: 
    cnt+=1
    lat = acs6.loc[i,'LATITUDE']
    lon = acs6.loc[i,'LONGITUDE']

    temp = cate1[cate1['poi_lat']>=lat-p_delta_degrees].copy()
    temp = temp[temp['poi_lat']<=lat+p_delta_degrees]
    temp = temp[temp['poi_lon']>=lon-p_delta_degrees]
    temp = temp[temp['poi_lon']<=lon+p_delta_degrees]
    #samp = sample(range(nbct), 200)
    for j in temp.index:
        xt,yt = Loc(lat,lon,temp.loc[j,'poi_lat'],temp.loc[j,'poi_lon'])
        line = [cnt,j,xt,yt,temp.loc[j,'poi_naics']]
        grid_S_random_I_cate1.append(line)
        
temp = np.array(grid_S_random_I_cate1) #
out = pd.DataFrame(data=temp, index=None, columns=['S_id','S_id_cate1','x','y','naics'])

# save data
out.to_csv(dataSaveDIR+'grid_S_random_I_cate1.csv' #
           ,index=False,header=out.columns,sep=',')


In [ ]:
cntr = 1000

grid_S_random_I_cate2 = [] #
cnt=cntr
for i in sampS: 
    cnt+=1
    lat = acs6.loc[i,'LATITUDE']
    lon = acs6.loc[i,'LONGITUDE']

    temp = cate2[cate2['poi_lat']>=lat-p_delta_degrees].copy()
    temp = temp[temp['poi_lat']<=lat+p_delta_degrees]
    temp = temp[temp['poi_lon']>=lon-p_delta_degrees]
    temp = temp[temp['poi_lon']<=lon+p_delta_degrees]
    #samp = sample(range(nbct), 200)
    for j in temp.index:
        xt,yt = Loc(lat,lon,temp.loc[j,'poi_lat'],temp.loc[j,'poi_lon'])
        line = [cnt,j,xt,yt,temp.loc[j,'poi_naics']]
        grid_S_random_I_cate2.append(line)
        
temp = np.array(grid_S_random_I_cate2) #
out = pd.DataFrame(data=temp, index=None, columns=['S_id','S_id_cate2','x','y','naics'])

# save data
out.to_csv(dataSaveDIR+'grid_S_random_I_cate2.csv' #
           ,index=False,header=out.columns,sep=',')

grid_S_random_I_cate3 = [] #
cnt=cntr
for i in sampS: 
    cnt+=1
    lat = acs6.loc[i,'LATITUDE']
    lon = acs6.loc[i,'LONGITUDE']

    temp = cate3[cate3['poi_lat']>=lat-p_delta_degrees].copy()
    temp = temp[temp['poi_lat']<=lat+p_delta_degrees]
    temp = temp[temp['poi_lon']>=lon-p_delta_degrees]
    temp = temp[temp['poi_lon']<=lon+p_delta_degrees]
    #samp = sample(range(nbct), 200)
    for j in temp.index:
        xt,yt = Loc(lat,lon,temp.loc[j,'poi_lat'],temp.loc[j,'poi_lon'])
        line = [cnt,j,xt,yt,temp.loc[j,'poi_naics']]
        grid_S_random_I_cate3.append(line)
        
temp = np.array(grid_S_random_I_cate3) #
out = pd.DataFrame(data=temp, index=None, columns=['S_id','S_id_cate3','x','y','naics'])

out.to_csv(dataSaveDIR+'grid_S_random_I_cate3.csv' #
           ,index=False,header=out.columns,sep=',')

grid_S_random_I_cate4 = [] #
cnt=cntr
for i in sampS:
    cnt+=1
    lat = acs6.loc[i,'LATITUDE']
    lon = acs6.loc[i,'LONGITUDE']

    temp = cate4[cate4['poi_lat']>=lat-p_delta_degrees].copy()
    temp = temp[temp['poi_lat']<=lat+p_delta_degrees]
    temp = temp[temp['poi_lon']>=lon-p_delta_degrees]
    temp = temp[temp['poi_lon']<=lon+p_delta_degrees]
    #samp = sample(range(nbct), 200)
    for j in temp.index:
        xt,yt = Loc(lat,lon,temp.loc[j,'poi_lat'],temp.loc[j,'poi_lon'])
        line = [cnt,j,xt,yt,temp.loc[j,'poi_naics']]
        grid_S_random_I_cate4.append(line)
        
temp = np.array(grid_S_random_I_cate4) #
out = pd.DataFrame(data=temp, index=None, columns=['S_id','S_id_cate4','x','y','naics'])

out.to_csv(dataSaveDIR+'grid_S_random_I_cate4.csv' #
           ,index=False,header=out.columns,sep=',')

grid_S_random_I_cate5 = [] #
cnt=cntr
for i in sampS: 
    cnt+=1
    lat = acs6.loc[i,'LATITUDE']
    lon = acs6.loc[i,'LONGITUDE']

    temp = cate5[cate5['poi_lat']>=lat-p_delta_degrees].copy()
    temp = temp[temp['poi_lat']<=lat+p_delta_degrees]
    temp = temp[temp['poi_lon']>=lon-p_delta_degrees]
    temp = temp[temp['poi_lon']<=lon+p_delta_degrees]
    #samp = sample(range(nbct), 200)
    for j in temp.index:
        xt,yt = Loc(lat,lon,temp.loc[j,'poi_lat'],temp.loc[j,'poi_lon'])
        line = [cnt,j,xt,yt,temp.loc[j,'poi_naics']]
        grid_S_random_I_cate5.append(line)
        
temp = np.array(grid_S_random_I_cate5) #
out = pd.DataFrame(data=temp, index=None, columns=['S_id','S_id_cate5','x','y','naics'])

out.to_csv(dataSaveDIR+'grid_S_random_I_cate5.csv' #
           ,index=False,header=out.columns,sep=',')

grid_S_random_I_comp = [] #
cnt=cntr
for i in sampS: 
    cnt+=1
    lat = acs6.loc[i,'LATITUDE']
    lon = acs6.loc[i,'LONGITUDE']

    temp = comp[comp['poi_lat']>=lat-p_delta_degrees].copy()
    temp = temp[temp['poi_lat']<=lat+p_delta_degrees]
    temp = temp[temp['poi_lon']>=lon-p_delta_degrees]
    temp = temp[temp['poi_lon']<=lon+p_delta_degrees]
    #samp = sample(range(nbct), 200)
    for j in temp.index:
        xt,yt = Loc(lat,lon,temp.loc[j,'poi_lat'],temp.loc[j,'poi_lon'])
        line = [cnt,j,xt,yt,temp.loc[j,'poi_naics']]
        grid_S_random_I_comp.append(line)
        
temp = np.array(grid_S_random_I_comp) #
out = pd.DataFrame(data=temp, index=None, columns=['S_id','S_id_comp','x','y','naics'])

out.to_csv(dataSaveDIR+'grid_S_random_I_comp.csv' #
           ,index=False,header=out.columns,sep=',')